In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
# Load metadata
metadata_path = 'HAM10000_metadata.csv'
metadata = pd.read_csv(metadata_path)
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [7]:
# Paths
images_folder = 'images'

In [9]:
# Preprocess the labels
label_encoder = LabelEncoder()
metadata['label'] = label_encoder.fit_transform(metadata['dx'])

In [11]:
# Load images and labels
image_size = (128, 128)  # Resize images to 128x128
images = []
labels = []

for idx, row in metadata.iterrows():
    image_path = os.path.join(images_folder, row['image_id'] + '.jpg')
    if os.path.exists(image_path):
        img = load_img(image_path, target_size=image_size)
        img_array = img_to_array(img) / 255.0  # Normalize pixel values
        images.append(img_array)
        labels.append(row['label'])

images = np.array(images)
labels = np.array(labels)

In [13]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [15]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

In [19]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(labels)), activation='softmax')
])


C:\Users\risku\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
# Train the model
epochs = 20
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

C:\Users\risku\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 205s 800ms/step - accuracy: 0.6597 - loss: 1.1375 - val_accuracy: 0.6680 - val_loss: 0.9511
Epoch 2/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 257s 1s/step - accuracy: 0.6663 - loss: 0.9796 - val_accuracy: 0.6680 - val_loss: 0.9761
Epoch 3/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 222s 884ms/step - accuracy: 0.6668 - loss: 0.9758 - val_accuracy: 0.6680 - val_loss: 0.9020
Epoch 4/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 266s 901ms/step - accuracy: 0.6707 - loss: 0.8932 - val_accuracy: 0.6705 - val_loss: 0.8529
Epoch 5/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 198s 786ms/step - accuracy: 0.6794 - loss: 0.8699 - val_accuracy: 0.6720 - val_loss: 0.8431
Epoch 6/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 202s 805ms/step - accuracy: 0.6810 - loss: 0.8531 - val_accuracy: 0.6795 - val_loss: 0.8264
Epoch 7/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 237s 944ms/step - accuracy: 0.6786 - loss: 0.8655 - val_accuracy: 0.6805 - val_loss: 0.7941
Epoch 8/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 210s 832ms/step - accuracy: 0.6876 - lo

In [25]:
# Save the model
model.save('SkinModel.h5')

print("Model training complete. 'SkinModel.h5' saved.")

Model training complete. 'SkinModel.h5' saved.
